In [2]:
!pip install pulp
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd

projectsCSV = pd.read_csv(r'C:\Users\johan\OneDrive\Schule\10. Klasse\Schülerprojektwoche\projects.csv', sep=';')
preferencesCSV = pd.read_csv(r'C:\Users\johan\OneDrive\Schule\10. Klasse\Schülerprojektwoche\votes.csv', sep=';')



# create a dictionary of the projects with a unique ID as key and the project name (first row) and the number of participants (third row) as value
projects = {i: {'title': projectsCSV.iloc[i][0], 'max': projectsCSV.iloc[i][2]} for i in range(len(projectsCSV))}

print(projects)

# Create preferences dictionary using project IDs instead of names
preferences = {
    i: [
        [key for key, value in projects.items() if value['title'] == row['1. Wahl']][0] if any(value['title'] == row['1. Wahl'] for value in projects.values()) else None,
        [key for key, value in projects.items() if value['title'] == row['2. Wahl']][0] if any(value['title'] == row['2. Wahl'] for value in projects.values()) else None,
        [key for key, value in projects.items() if value['title'] == row['3. Wahl']][0] if any(value['title'] == row['3. Wahl'] for value in projects.values()) else None
    ]
    for i, row in preferencesCSV.iterrows()
}

print(projects)
print(preferences)

{0: {'title': 'Wir vs. Müll', 'max': np.int64(15)}, 1: {'title': 'Kreatives Schreiben', 'max': np.int64(10)}, 2: {'title': 'Volleyball', 'max': np.int64(12)}, 3: {'title': 'Musik-Produktion am PC', 'max': np.int64(10)}, 4: {'title': 'Vegan kochen & backen', 'max': np.int64(12)}, 5: {'title': 'Schmuck mit Perlen', 'max': np.int64(12)}, 6: {'title': 'Die Welt – was sie bewegt und wem sie gehört', 'max': np.int64(15)}, 7: {'title': 'Fußball', 'max': np.int64(22)}, 8: {'title': 'Bouldern', 'max': np.int64(10)}, 9: {'title': 'Soziale Projekte', 'max': np.int64(8)}, 10: {'title': 'Beseeltes Schreiben', 'max': np.int64(15)}}
{0: {'title': 'Wir vs. Müll', 'max': np.int64(15)}, 1: {'title': 'Kreatives Schreiben', 'max': np.int64(10)}, 2: {'title': 'Volleyball', 'max': np.int64(12)}, 3: {'title': 'Musik-Produktion am PC', 'max': np.int64(10)}, 4: {'title': 'Vegan kochen & backen', 'max': np.int64(12)}, 5: {'title': 'Schmuck mit Perlen', 'max': np.int64(12)}, 6: {'title': 'Die Welt – was sie bewe

C:\Users\johan\AppData\Local\Temp\ipykernel_23500\136023282.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  projects = {i: {'title': projectsCSV.iloc[i][0], 'max': projectsCSV.iloc[i][2]} for i in range(len(projectsCSV))}


In [11]:
import pulp

# Anzahl der Teilnehmer und Kurse
num_participants = len(preferences)
num_courses = len(projects)

# Scores für Präferenzen: 1 für erste Wahl, 2 für zweite Wahl, 4 für dritte Wahl
scores = {
    'first': 1,
    'second': 2,
    'third': 4
}

# LP-Problem erstellen
problem = pulp.LpProblem("CourseAssignment", pulp.LpMinimize)

# Entscheidungsvariablen: x_ij ist 1, wenn Teilnehmer i Kurs j zugewiesen wird, sonst 0
x = pulp.LpVariable.dicts("x", ((i, j) for i in range(num_participants) for j in range(num_courses)), cat="Binary")

# Überbuchungsvariablen: o_j ist 0 oder mehr, wenn Kurs j überbucht ist
o = pulp.LpVariable.dicts("o", (j for j in range(num_courses)), lowBound=0, cat="Integer")

# Zielfunktion: Minimieren der Präferenz-Scores und Überbuchungsstrafen
problem += pulp.lpSum(
    scores['first'] * x[i, preferences[i][0]] +
    scores['second'] * x[i, preferences[i][1]] +
    scores['third'] * x[i, preferences[i][2]]
    for i in range(num_participants)
) + pulp.lpSum(10 * o[j] for j in range(num_courses))

# Nebenbedingung: Jeder Teilnehmer wird genau einem Kurs zugewiesen
for i in range(num_participants):
    problem += pulp.lpSum(x[i, j] for j in range(num_courses)) == 1

# Nebenbedingung: Maximale Teilnehmeranzahl pro Kurs (mit Überbuchung)
for j in range(num_courses):
    problem += pulp.lpSum(x[i, j] for i in range(num_participants)) <= projects[j]['max'] + o[j]

# Präferenzzuweisung: Teilnehmer dürfen nur auf ihre 1., 2. oder 3. Wahl zugewiesen werden
for i in range(num_participants):
    for j in range(num_courses):
        if j not in preferences[i]:
            problem += x[i, j] == 0

# Problem lösen
problem.solve()

print(num_participants)
print(num_courses)
print(scores)

106
11
{'first': 1, 'second': 2, 'third': 4}


In [12]:
# Ergebnisse ausgeben
print("Status:", pulp.LpStatus[problem.status])

for i in range(num_participants):
    for j in range(num_courses):
        if pulp.value(x[i, j]) == 1:
            if j == preferences[i][0]:
                wahl = '1. Wahl'
            elif j == preferences[i][1]:
                wahl = '2. Wahl'
            elif j == preferences[i][2]:
                wahl = '3. Wahl'
            print(f"Teilnehmer {i} wurde Kurs {j} zugewiesen ({wahl})")

for j in range(num_courses):
    print(f"Überbuchung für Kurs {j}: {pulp.value(o[j])}")

# Anzahl der Teilnehmer in jedem Kurs ausgeben
for j in range(num_courses):
    teilnehmer_in_kurs = sum(pulp.value(x[i, j]) for i in range(num_participants))
    if j in projects:
        print(f"Kurs {projects[j]} hat {teilnehmer_in_kurs} Teilnehmer")
    else:
        print(f"Kurs {j} hat {teilnehmer_in_kurs} Teilnehmer")

# Initialize counts for first, second, and third choices in each project
for project in projects.values():
    project['first'] = 0
    project['second'] = 0
    project['third'] = 0

# Anzahl an 1. 2. und 3. Wahlen ausgeben
for i in range(num_participants):
    for j in range(num_courses):
        if pulp.value(x[i, j]) == 1:
            if j == preferences[i][0]:
                projects[j]['first'] += 1
            elif j == preferences[i][1]:
                projects[j]['second'] += 1
            elif j == preferences[i][2]:
                projects[j]['third'] += 1

# Print the counts for each project
for j, project in projects.items():
    print(f"Projekt {project['title']}: 1. Wahl: {project['first']}, 2. Wahl: {project['second']}, 3. Wahl: {project['third']}")

# print the counts in total
first_choices = sum(project['first'] for project in projects.values())
second_choices = sum(project['second'] for project in projects.values())
third_choices = sum(project['third'] for project in projects.values())
print(f"1. Wahl: {first_choices}, 2. Wahl: {second_choices}, 3. Wahl: {third_choices}")

# Print the total number of participants in each project
for j, project in projects.items():
    print(f"Projekt {project['title']}: {sum(pulp.value(x[i, j]) for i in range(num_participants))} Teilnehmer")

Status: Optimal
Teilnehmer 0 wurde Kurs 8 zugewiesen (2. Wahl)
Teilnehmer 1 wurde Kurs 6 zugewiesen (2. Wahl)
Teilnehmer 2 wurde Kurs 5 zugewiesen (1. Wahl)
Teilnehmer 3 wurde Kurs 0 zugewiesen (3. Wahl)
Teilnehmer 4 wurde Kurs 4 zugewiesen (1. Wahl)
Teilnehmer 5 wurde Kurs 7 zugewiesen (1. Wahl)
Teilnehmer 6 wurde Kurs 0 zugewiesen (3. Wahl)
Teilnehmer 7 wurde Kurs 2 zugewiesen (1. Wahl)
Teilnehmer 8 wurde Kurs 7 zugewiesen (1. Wahl)
Teilnehmer 9 wurde Kurs 3 zugewiesen (1. Wahl)
Teilnehmer 10 wurde Kurs 4 zugewiesen (1. Wahl)
Teilnehmer 11 wurde Kurs 6 zugewiesen (2. Wahl)
Teilnehmer 12 wurde Kurs 4 zugewiesen (1. Wahl)
Teilnehmer 13 wurde Kurs 3 zugewiesen (2. Wahl)
Teilnehmer 14 wurde Kurs 10 zugewiesen (1. Wahl)
Teilnehmer 15 wurde Kurs 8 zugewiesen (1. Wahl)
Teilnehmer 16 wurde Kurs 7 zugewiesen (2. Wahl)
Teilnehmer 17 wurde Kurs 6 zugewiesen (2. Wahl)
Teilnehmer 18 wurde Kurs 5 zugewiesen (1. Wahl)
Teilnehmer 19 wurde Kurs 1 zugewiesen (3. Wahl)
Teilnehmer 20 wurde Kurs 5 zugewi